In [ ]:
# !pip install langchain==0.1.0 openai==1.7.2 langchain-openai==0.0.2 langchain-community==0.0.12 langchainhub==0.1.14
# !pip install chromadb==0.5.5

In [ ]:
# !pip install langchain_openai
# !pip install langchain
# !pip install -U langchain-community

'0.1.0'

https://python.langchain.com/v0.1/docs/use_cases/chatbots/#chat-retrieval

## Basic

In [ ]:
from google.colab import userdata
OPEN_API_KEY = userdata.get('OPENAI_API_KEY')

In [ ]:
import langchain
from langchain_openai import ChatOpenAI
from langchain.schema.messages import HumanMessage, SystemMessage
from langchain.prompts import ChatPromptTemplate
from langchain.prompts import (
    PromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    ChatPromptTemplate,
)
from langchain_openai import ChatOpenAI
from langchain.prompts import (
    PromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    ChatPromptTemplate,
)


In [ ]:
chat_model = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0, openai_api_key = OPEN_API_KEY)

messages = [
     SystemMessage(
         content="""You're an assistant knowledgeable about
         healthcare. Only answer healthcare-related questions."""
     ),
     HumanMessage(content="What is Medicaid managed care?"),
]
chat_model.invoke(messages)

AIMessage(content='Medicaid managed care is a system in which states contract with managed care organizations (MCOs) to provide healthcare services to Medicaid beneficiaries. These MCOs are responsible for coordinating and delivering healthcare services to enrollees in exchange for a fixed monthly payment per enrollee. Medicaid managed care aims to improve access to care, enhance quality, and control costs for Medicaid beneficiaries.')

In [ ]:
messages3 = [
     SystemMessage(
         content="""You're an assistant knowledgeable about
         politics. DO NOT answer any questions about Chinese politics."""
     ),
     HumanMessage(content="Tell me about Xi Jinping?"),
]
chat_model.invoke(messages3)

AIMessage(content="I'm sorry, but I'm unable to provide information on Chinese politics. If you have any other questions about politics in general, feel free to ask!")

In [ ]:
review_system_template_str = """Your job is to use patient
reviews to answer questions about their experience at a
hospital. Use the following context to answer questions.
Be as detailed as possible, but don't make up any information
that's not from the context. If you don't know an answer, say
you don't know.

{context}
"""

review_system_prompt = SystemMessagePromptTemplate(
    prompt=PromptTemplate(
        input_variables=["context"], template=review_system_template_str
    )
)

review_human_prompt = HumanMessagePromptTemplate(
    prompt=PromptTemplate(
        input_variables=["question"], template="{question}"
    )
)

messages = [review_system_prompt, review_human_prompt]
review_prompt_template = ChatPromptTemplate(
    input_variables=["context", "question"],
    messages=messages,
)
context = "Bob had a great stay!"
question = "Did anyone have a positive experience?"

review_prompt_template.format_messages(context=context, question=question)

review_chain = review_prompt_template | chat_model
review_chain.invoke({"context": context, "question": question})

AIMessage(content='Yes, Bob had a great stay at the hospital.')

## RAG

In [ ]:
# !pip install chromadb

In [ ]:
from langchain.document_loaders.csv_loader import CSVLoader
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings

REVIEWS_CSV_PATH = "./sample_data/reviews.csv"
REVIEWS_CHROMA_PATH = "chroma_data/"

loader = CSVLoader(file_path=REVIEWS_CSV_PATH, source_column="review")
reviews = loader.load()

reviews_vector_db = Chroma.from_documents(
    reviews, OpenAIEmbeddings(openai_api_key = OPEN_API_KEY), persist_directory=REVIEWS_CHROMA_PATH
)

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.prompts import (
    PromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    ChatPromptTemplate,
)
from langchain_core.output_parsers import StrOutputParser
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain.schema.runnable import RunnablePassthrough

REVIEWS_CHROMA_PATH = "chroma_data/"

# ...

reviews_retriever  = reviews_vector_db.as_retriever(k=10)

question = """Has anyone complained about
           communication with the hospital staff?"""
relevant_docs = reviews_vector_db.similarity_search(question, k=3)

print(relevant_docs[0].page_content)
print(relevant_docs[1].page_content)
print(relevant_docs[2].page_content)


review_id: 73
visit_id: 7696
review: I had a frustrating experience at the hospital. The communication between the medical staff and me was unclear, leading to misunderstandings about my treatment plan. Improvement is needed in this area.
physician_name: Maria Thompson
hospital_name: Little-Spencer
patient_name: Terri Smith
review_id: 73
visit_id: 7696
review: I had a frustrating experience at the hospital. The communication between the medical staff and me was unclear, leading to misunderstandings about my treatment plan. Improvement is needed in this area.
physician_name: Maria Thompson
hospital_name: Little-Spencer
patient_name: Terri Smith
review_id: 73
visit_id: 7696
review: I had a frustrating experience at the hospital. The communication between the medical staff and me was unclear, leading to misunderstandings about my treatment plan. Improvement is needed in this area.
physician_name: Maria Thompson
hospital_name: Little-Spencer
patient_name: Terri Smith


In [ ]:
from langchain_openai import ChatOpenAI
from langchain.prompts import (
    PromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    ChatPromptTemplate,
)
from langchain_core.output_parsers import StrOutputParser
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain.schema.runnable import RunnablePassthrough

REVIEWS_CHROMA_PATH = "chroma_data/"

# ...

reviews_vector_db = Chroma(
    persist_directory=REVIEWS_CHROMA_PATH,
    embedding_function=OpenAIEmbeddings(openai_api_key = OPEN_API_KEY)
)

reviews_retriever  = reviews_vector_db.as_retriever(k=10)

review_chain = (
    {"context": reviews_retriever, "question": RunnablePassthrough()}
    | review_prompt_template
    | chat_model
    | StrOutputParser()
)

In [ ]:
question = """Has anyone complained about
           communication with the hospital staff?"""
review_chain.invoke(question)

'Yes, a patient named Terri Smith complained about the communication between the medical staff and herself at the hospital. She mentioned that the communication was unclear, leading to misunderstandings about her treatment plan. Terri Smith suggested that improvement is needed in this area.'

In [ ]:
question2 = """what's my previous question?"""
review_chain.invoke(question2)

'Your previous question was: "What\'s my previous question?"'

In [ ]:
from langchain_core.messages import SystemMessage
from langchain_core.prompts import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
)
from langchain_openai import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain

# prompt
review_system_prompt = SystemMessagePromptTemplate(
    prompt=PromptTemplate(
        input_variables=["context"], template=review_system_template_str
    )
)

review_human_prompt = HumanMessagePromptTemplate(
    prompt=PromptTemplate(
        input_variables=["question"], template="{question}"
    )
)
messages = [review_system_prompt, review_human_prompt]

review_prompt_template = ChatPromptTemplate(
    input_variables=["context", "question"],
    messages=messages,
)

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

qa_chain = ConversationalRetrievalChain.from_llm(
    chat_model,
    reviews_retriever,
    memory=memory,
    verbose=False,
    combine_docs_chain_kwargs={"prompt": review_prompt_template},
)

question = "Has anyone complained about communication with the hospital staff?"

qa_chain.invoke({"question": question})




{'question': 'Has anyone complained about communication with the hospital staff?',
 'chat_history': [HumanMessage(content='Has anyone complained about communication with the hospital staff?'),
  AIMessage(content='Yes, Terri Smith has complained about the communication between the medical staff and herself at the hospital. She mentioned that the unclear communication led to misunderstandings about her treatment plan and expressed the need for improvement in this area.')],
 'answer': 'Yes, Terri Smith has complained about the communication between the medical staff and herself at the hospital. She mentioned that the unclear communication led to misunderstandings about her treatment plan and expressed the need for improvement in this area.'}

In [ ]:
question2 = "what's my previous question?"

qa_chain.invoke({"question": question2})

{'question': "what's my previous question?",
 'chat_history': [HumanMessage(content='Has anyone complained about communication with the hospital staff?'),
  AIMessage(content='Yes, Terri Smith has complained about the communication between the medical staff and herself at the hospital. She mentioned that the unclear communication led to misunderstandings about her treatment plan and expressed the need for improvement in this area.'),
  HumanMessage(content="what's my previous question?"),
  AIMessage(content="The previous question was asking about the patient's experience at the hospital, specifically regarding the medical care received, the hospital facilities, and the communication about the daily schedule and procedures.")],
 'answer': "The previous question was asking about the patient's experience at the hospital, specifically regarding the medical care received, the hospital facilities, and the communication about the daily schedule and procedures."}

## Agent

In [ ]:
import random
import time

def get_current_wait_time(hospital: str) -> int | str:
    """Dummy function to generate fake wait times"""

    if hospital not in ["A", "B", "C", "D"]:
        return f"Hospital {hospital} does not exist"

    # Simulate API call delay
    time.sleep(1)

    return random.randint(0, 10000)

In [ ]:
# !pip install langchainhub

In [ ]:
import dotenv
from langchain_openai import ChatOpenAI
from langchain.prompts import (
    PromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    ChatPromptTemplate,
)
from langchain_core.output_parsers import StrOutputParser
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain.schema.runnable import RunnablePassthrough
from langchain.agents import (
    create_openai_functions_agent,
    Tool,
    AgentExecutor,
)
from langchain import hub

# ...

tools = [
    Tool(
        name="Reviews",
        func=review_chain.invoke,
        description="""Useful when you need to answer questions
        about patient reviews or experiences at the hospital.
        Not useful for answering questions about specific visit
        details such as payer, billing, treatment, diagnosis,
        chief complaint, hospital, or physician information.
        Pass the entire question as input to the tool. For instance,
        if the question is "What do patients think about the triage system?",
        the input should be "What do patients think about the triage system?"
        """,
    ),
    Tool(
        name="Waits",
        func=get_current_wait_time,
        description="""Use when asked about current wait times
        at a specific hospital. This tool can only get the current
        wait time at a hospital and does not have any information about
        aggregate or historical wait times. This tool returns wait times in
        minutes. Do not pass the word "hospital" as input,
        only the hospital name itself. For instance, if the question is
        "What is the wait time at hospital A?", the input should be "A".
        """,
    ),
]

hospital_agent_prompt = hub.pull("hwchase17/openai-functions-agent")

agent_chat_model = ChatOpenAI(
    model="gpt-3.5-turbo-1106",
    temperature=0,
    openai_api_key = OPEN_API_KEY,
)

hospital_agent = create_openai_functions_agent(
    llm=agent_chat_model,
    prompt=hospital_agent_prompt,
    tools=tools,
)

hospital_agent_executor = AgentExecutor(
    agent=hospital_agent,
    tools=tools,
    return_intermediate_steps=True,
    verbose=True,
)

In [ ]:
hospital_agent_executor.invoke(
    {"input": "What is the current wait time at hospital C?"}
)

hospital_agent_executor.invoke(
    {"input": "What have patients said about their comfort at the hospital?"}
)




> Entering new AgentExecutor chain...

Invoking: `Waits` with `C`


4359The current wait time at hospital C is 43 minutes and 59 seconds.

> Finished chain.


> Entering new AgentExecutor chain...

Invoking: `Reviews` with `What have patients said about their comfort at the hospital?`


Patients have mentioned that the hospital's dedication to patient comfort was evident in well-designed private rooms and comfortable furnishings. This made their recovery more bearable and contributed to an overall positive experience. However, one patient did mention that the uncomfortable beds made it difficult to get a good night's sleep during their stay.Patients have mentioned that the hospital's dedication to patient comfort was evident in well-designed private rooms and comfortable furnishings. This made their recovery more bearable and contributed to an overall positive experience. However, one patient did mention that the uncomfortable beds made it difficult to get a good night's sleep during

{'input': 'What have patients said about their comfort at the hospital?',
 'output': "Patients have mentioned that the hospital's dedication to patient comfort was evident in well-designed private rooms and comfortable furnishings. This made their recovery more bearable and contributed to an overall positive experience. However, one patient did mention that the uncomfortable beds made it difficult to get a good night's sleep during their stay.",
 'intermediate_steps': [(AgentActionMessageLog(tool='Reviews', tool_input='What have patients said about their comfort at the hospital?', log='\nInvoking: `Reviews` with `What have patients said about their comfort at the hospital?`\n\n\n', message_log=[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"__arg1":"What have patients said about their comfort at the hospital?"}', 'name': 'Reviews'}})]),
   "Patients have mentioned that the hospital's dedication to patient comfort was evident in well-designed private rooms an

## PDF Chatbot Example

In [ ]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain.vectorstores import DocArrayInMemorySearch
from langchain.document_loaders import TextLoader
from langchain.chains import RetrievalQA,  ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import TextLoader
from langchain.document_loaders import PyPDFLoader

In [ ]:
llm_name = "gpt-3.5-turbo-0125"
def load_db(file, chain_type, k):
    # load documents
    loader = PyPDFLoader(file)
    documents = loader.load()
    # split documents
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=150)
    docs = text_splitter.split_documents(documents)
    # define embedding
    embeddings = OpenAIEmbeddings(openai_api_key = OPEN_API_KEY)
    # create vector database from data
    db = DocArrayInMemorySearch.from_documents(docs, embeddings)
    # define retriever
    retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": k})
    # create a chatbot chain. Memory is managed externally.
    qa = ConversationalRetrievalChain.from_llm(
        llm=ChatOpenAI(model_name=llm_name, temperature=0, openai_api_key = OPEN_API_KEY,),
        chain_type=chain_type,
        retriever=retriever,
        return_source_documents=True,
        return_generated_question=True,
    )
    return qa

In [ ]:
import panel as pn
import param

class cbfs(param.Parameterized):
    chat_history = param.List([])
    answer = param.String("")
    db_query  = param.String("")
    db_response = param.List([])

    def __init__(self,  **params):
        super(cbfs, self).__init__( **params)
        self.panels = []
        self.loaded_file = "sample_data/MachineLearning-Lecture01.pdf"
        self.qa = load_db(self.loaded_file,"stuff", 4)

    def call_load_db(self, count):
        if count == 0 or file_input.value is None:  # init or no file specified :
            return pn.pane.Markdown(f"Loaded File: {self.loaded_file}")
        else:
            file_input.save("temp.pdf")  # local copy
            self.loaded_file = file_input.filename
            button_load.button_style="outline"
            self.qa = load_db("temp.pdf", "stuff", 4)
            button_load.button_style="solid"
        self.clr_history()
        return pn.pane.Markdown(f"Loaded File: {self.loaded_file}")

    def convchain(self, query):
        if not query:
            return pn.WidgetBox(pn.Row('User:', pn.pane.Markdown("", width=600)), scroll=True)
        result = self.qa({"question": query, "chat_history": self.chat_history})
        self.chat_history.extend([(query, result["answer"])])
        self.db_query = result["generated_question"]
        self.db_response = result["source_documents"]
        self.answer = result['answer']
        self.panels.extend([
            pn.Row('User:', pn.pane.Markdown(query, width=600)),
            pn.Row('ChatBot:', pn.pane.Markdown(self.answer, width=600, styles={'background-color': '#F6F6F6'}))
        ])
        inp.value = ''  #clears loading indicator when cleared
        return pn.WidgetBox(*self.panels,scroll=True)

    @param.depends('db_query ', )
    def get_lquest(self):
        if not self.db_query :
            return pn.Column(
                pn.Row(pn.pane.Markdown(f"Last question to DB:", styles={'background-color': '#F6F6F6'})),
                pn.Row(pn.pane.Str("no DB accesses so far"))
            )
        return pn.Column(
            pn.Row(pn.pane.Markdown(f"DB query:", styles={'background-color': '#F6F6F6'})),
            pn.pane.Str(self.db_query )
        )

    @param.depends('db_response', )
    def get_sources(self):
        if not self.db_response:
            return
        rlist=[pn.Row(pn.pane.Markdown(f"Result of DB lookup:", styles={'background-color': '#F6F6F6'}))]
        for doc in self.db_response:
            rlist.append(pn.Row(pn.pane.Str(doc)))
        return pn.WidgetBox(*rlist, width=600, scroll=True)

    @param.depends('convchain', 'clr_history')
    def get_chats(self):
        if not self.chat_history:
            return pn.WidgetBox(pn.Row(pn.pane.Str("No History Yet")), width=600, scroll=True)
        rlist=[pn.Row(pn.pane.Markdown(f"Current Chat History variable", styles={'background-color': '#F6F6F6'}))]
        for exchange in self.chat_history:
            rlist.append(pn.Row(pn.pane.Str(exchange)))
        return pn.WidgetBox(*rlist, width=600, scroll=True)

    def clr_history(self,count=0):
        self.chat_history = []
        return


In [ ]:
# !pip install pypdf
# !pip install -U docarray
# !pip install jupyter_bokeh

In [ ]:
cb = cbfs()

file_input = pn.widgets.FileInput(accept='.pdf')
button_load = pn.widgets.Button(name="Load DB", button_type='primary')
button_clearhistory = pn.widgets.Button(name="Clear History", button_type='warning')
button_clearhistory.on_click(cb.clr_history)
inp = pn.widgets.TextInput( placeholder='Enter text here…')

bound_button_load = pn.bind(cb.call_load_db, button_load.param.clicks)
conversation = pn.bind(cb.convchain, inp)

# jpg_pane = pn.pane.Image( './img/convchain.jpg')

tab1 = pn.Column(
    pn.Row(inp),
    pn.layout.Divider(),
    pn.panel(conversation,  loading_indicator=True, height=300),
    pn.layout.Divider(),
)
tab2= pn.Column(
    pn.panel(cb.get_lquest),
    pn.layout.Divider(),
    pn.panel(cb.get_sources ),
)
tab3= pn.Column(
    pn.panel(cb.get_chats),
    pn.layout.Divider(),
)
tab4=pn.Column(
    pn.Row( file_input, button_load, bound_button_load),
    pn.Row( button_clearhistory, pn.pane.Markdown("Clears chat history. Can use to start a new topic" )),
    pn.layout.Divider(),
    # pn.Row(jpg_pane.clone(width=400))
)
dashboard = pn.Column(
    pn.Row(pn.pane.Markdown('# ChatWithYourData_Bot')),
    pn.Tabs(('Conversation', tab1), ('Database', tab2), ('Chat History', tab3),('Configure', tab4))
)
pn.extension()
dashboard

Traceback (most recent call last):
 File "/usr/local/lib/python3.10/dist-packages/pyviz_comms/__init__.py", line 340, in _handle_msg
 self._on_msg(msg)
 File "/usr/local/lib/python3.10/dist-packages/panel/viewable.py", line 478, in _on_msg
 doc.unhold()
 File "/usr/local/lib/python3.10/dist-packages/bokeh/document/document.py", line 776, in unhold
 self.callbacks.unhold()
 File "/usr/local/lib/python3.10/dist-packages/bokeh/document/callbacks.py", line 432, in unhold
 self.trigger_on_change(event)
 File "/usr/local/lib/python3.10/dist-packages/bokeh/document/callbacks.py", line 409, in trigger_on_change
 invoke_with_curdoc(doc, event.callback_invoker)
 File "/usr/local/lib/python3.10/dist-packages/bokeh/document/callbacks.py", line 444, in invoke_with_curdoc
 return f()
 File "/usr/local/lib/python3.10/dist-packages/bokeh/util/callback_manager.py", line 185, in invoke
 callback(attr, old, new)
 File "/usr/local/lib/python3.10/dist-packages/panel/reactive.py", line 474, in _comm_change
 state._handle_exception(e)
 File "/usr/local/lib/python3.10/dist-packages/panel/io/state.py", line 458, in _handle_exception
 raise exception
 File "/usr/local/lib/python3.10/dist-packages/panel/reactive.py", line 472, in _comm_change
 self._schedule_change(doc, comm)
 File "/usr/local/lib/python3.10/dist-packages/panel/reactive.py", line 454, in _schedule_change
 self._change_event(doc)
 File "/usr/local/lib/python3.10/dist-packages/panel/reactive.py", line 450, in _change_event
 self._process_events(events)
 File "/usr/local/lib/python3.10/dist-packages/panel/reactive.py", line 387, in _process_events
 self.param.update(**self_params)
 File "/usr/local/lib/python3.10/dist-packages/param/parameterized.py", line 2319, in update
 restore = dict(self_._update(arg, **kwargs))
 File "/usr/local/lib/python3.10/dist-packages/param/parameterized.py", line 2352, in _update
 self_._batch_call_watchers()
 File "/usr/local/lib/python3.10/dist-packages/param/parameterized.py", line 2546, in _batch_call_watchers
 self_._execute_watcher(watcher, events)
 File "/usr/local/lib/python3.10/dist-packages/param/parameterized.py", line 2506, in _execute_watcher
 watcher.fn(*args, **kwargs)
 File "/usr/local/lib/python3.10/dist-packages/panel/param.py", line 882, in _replace_pane
 new_object = self.eval(self.object)
 File "/usr/local/lib/python3.10/dist-packages/panel/param.py", line 1097, in eval
 return eval_function_with_deps(ref)
 File "/usr/local/lib/python3.10/dist-packages/param/parameterized.py", line 165, in eval_function_with_deps
 return function(*args, **kwargs)
 File "/usr/local/lib/python3.10/dist-packages/param/depends.py", line 53, in _depends
 return func(*args, **kw)
 File "/usr/local/lib/python3.10/dist-packages/param/reactive.py", line 594, in wrapped
 return eval_fn()(*combined_args, **combined_kwargs)
 File "<ipython-input-24-75f88e07b577>", line 31, in convchain
 result = self.qa({"question": query, "chat_history": self.chat_history})
 File "/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py", line 145, in warning_emitting_wrapper
 return wrapped(*args, **kwargs)
 File "/usr/local/lib/python3.10/dist-packages/langchain/chains/base.py", line 363, in __call__
 return self.invoke(
 File "/usr/local/lib/python3.10/dist-packages/langchain/chains/base.py", line 162, in invoke
 raise e
 File "/usr/local/lib/python3.10/dist-packages/langchain/chains/base.py", line 156, in invoke
 self._call(inputs, run_manager=run_manager)
 File "/usr/local/lib/python3.10/dist-packages/langchain/chains/conversational_retrieval/base.py", line 155, in _call
 docs = self._get_docs(new_question, inputs, run_manager=_run_manager)
 File "/usr/local/lib/python3.10/dist-packages/langchain/chains/conversational_retrieval/base.py", line 317, in _get_docs
 docs = self.retriever.get_relevant_documents(
 File "/usr/local/lib/python3.10/dist-packages/langchain_core/retrievers.py", line 224, in get_relevant_documents
 raise e
 File "/usr/local/lib/python3.10/d

Column
    [0] Row
        [0] Markdown(str)
    [1] Tabs
        [0] Column
            [0] Row
                [0] TextInput(placeholder='Enter text here…')
            [1] Divider()
            [2] ParamFunction(function, _pane=WidgetBox, defer_load=False, height=300, loading_indicator=True)
            [3] Divider()
        [1] Column
            [0] ParamMethod(method, _pane=Column, defer_load=False)
            [1] Divider()
            [2] ParamMethod(method, _pane=Str, defer_load=False)
        [2] Column
            [0] ParamMethod(method, _pane=WidgetBox, defer_load=False)
            [1] Divider()
        [3] Column
            [0] Row
                [0] FileInput(accept='.pdf')
                [1] Button(button_type='primary', name='Load DB')
                [2] ParamFunction(function, _pane=Markdown, defer_load=False)
            [1] Row
                [0] Button(button_type='warning', name='Clear History')
                [1] Markdown(str)
            [2] Divider()

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


2024-08-02 07:01:24,056 ERROR: panel.reactive - Callback failed for object named '' changing property {'value': "what's the prerequisite of this course"} 
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/panel/reactive.py", line 387, in _process_events
    self.param.update(**self_params)
  File "/usr/local/lib/python3.10/dist-packages/param/parameterized.py", line 2319, in update
    restore = dict(self_._update(arg, **kwargs))
  File "/usr/local/lib/python3.10/dist-packages/param/parameterized.py", line 2352, in _update
    self_._batch_call_watchers()
  File "/usr/local/lib/python3.10/dist-packages/param/parameterized.py", line 2546, in _batch_call_watchers
    self_._execute_watcher(watcher, events)
  File "/usr/local/lib/python3.10/dist-packages/param/parameterized.py", line 2506, in _execute_watcher
    watcher.fn(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/panel/param.py", line 882, in _replace_pane
    new_object = self.eva